In [1]:
import json, pathlib
from collections import defaultdict

NEW_COMPLAINT_TYPES = {
    "category",
    "appearance",
    "handle",
    "collision",
    "joint",
    "metalink",
    "unknown",
}

complaints_by_model = defaultdict(list)
complaints_by_type = defaultdict(list)
for path in pathlib.Path(r"D:\ig_pipeline").glob("qa-2025/*/*/*.json"):
    try:
        batch = path.parts[-3]
        mdl = path.stem
        complaints = json.loads(path.read_text())["complaints"]

        # Update all of the QC: complaints to match the author's batch
        for c in complaints:
            c_question = c["message"]
            if c_question.startswith("QC: "):
                c["complaint"] = f"quick complaint added during qa-2025 batch {batch}"

        unresolved_complaints = [x for x in complaints if not x["processed"]]
        complaints_by_model[mdl].extend(unresolved_complaints)

        for c in unresolved_complaints:
            c_question = c["message"]
            if c_question.startswith("QC: "):
                c_type = c_question[4:]
                assert c_type in NEW_COMPLAINT_TYPES
            else:
                c_type = "legacy"
            complaints_by_type[c_type].append(c)
    except:
        print(f"Error processing {path}")

Error processing D:\ig_pipeline\qa-2025\qa-2025-hang\shelf\kxvbui.json


In [12]:
complained_objects = [x for x in complaints_by_model if len(complaints_by_model[x]) > 0]
print(len(complained_objects))

2055


In [13]:
print({k: len(v) for k, v in complaints_by_type.items()})

{'appearance': 773, 'metalink': 222, 'collision': 753, 'joint': 42, 'legacy': 346, 'category': 260, 'unknown': 77, 'handle': 63}
